In [267]:
import math
import numpy as np
import pandas as pd
import scipy
import seaborn as sn
sn.set(rc={'figure.figsize':(11, 4)})

from scipy import stats, optimize
from scipy.stats import genextreme
import matplotlib
import matplotlib.pyplot as plt
import scipy.stats
from sklearn import svm
from sklearn.preprocessing import MinMaxScaler
%matplotlib notebook

In [268]:
data = pd.read_csv('cs1_L95.csv')

In [269]:
data

,Unnamed: 0,runoff_obs,basin_cfs_pred,Id,year,month,day,precip,tmax,tmin
0,0,2.7,3.890482,1,2002,10,1,0.13,57.4,34.6
1,1,2.7,3.041588,2,2002,10,2,0.97,38.4,29.7
2,2,2.8,2.796749,3,2002,10,3,0.04,41.2,31.7
3,3,2.8,2.600715,4,2002,10,4,0.00,52.9,34.0
4,4,2.6,2.439560,5,2002,10,5,0.00,50.5,34.8
...,...,...,...,...,...,...,...,...,...,...
3460,3648,4.4,5.014700,3649,2012,9,26,0.00,59.2,41.0
3461,3649,4.3,4.979207,3650,2012,9,27,0.00,64.1,43.2
3462,3650,4.2,5.001390,3651,2012,9,28,0.00,68.0,46.2
3463,3651,4.2,5.114594,3652,2012,9,29,0.00,69.8,50.9


In [270]:
from datetime import date

In [271]:
data = data.assign(Date=pd.to_datetime(data[['year', 'month','day']]))

In [272]:
data

,Unnamed: 0,runoff_obs,basin_cfs_pred,Id,year,month,day,precip,tmax,tmin,Date
0,0,2.7,3.890482,1,2002,10,1,0.13,57.4,34.6,2002-10-01
1,1,2.7,3.041588,2,2002,10,2,0.97,38.4,29.7,2002-10-02
2,2,2.8,2.796749,3,2002,10,3,0.04,41.2,31.7,2002-10-03
3,3,2.8,2.600715,4,2002,10,4,0.00,52.9,34.0,2002-10-04
4,4,2.6,2.439560,5,2002,10,5,0.00,50.5,34.8,2002-10-05
...,...,...,...,...,...,...,...,...,...,...,...
3460,3648,4.4,5.014700,3649,2012,9,26,0.00,59.2,41.0,2012-09-26
3461,3649,4.3,4.979207,3650,2012,9,27,0.00,64.1,43.2,2012-09-27
3462,3650,4.2,5.001390,3651,2012,9,28,0.00,68.0,46.2,2012-09-28
3463,3651,4.2,5.114594,3652,2012,9,29,0.00,69.8,50.9,2012-09-29


In [273]:
data.drop(data.columns[[0,2,3,4,5,6]], axis=1, inplace=True)

In [274]:
data

,runoff_obs,precip,tmax,tmin,Date
0,2.7,0.13,57.4,34.6,2002-10-01
1,2.7,0.97,38.4,29.7,2002-10-02
2,2.8,0.04,41.2,31.7,2002-10-03
3,2.8,0.00,52.9,34.0,2002-10-04
4,2.6,0.00,50.5,34.8,2002-10-05
...,...,...,...,...,...
3460,4.4,0.00,59.2,41.0,2012-09-26
3461,4.3,0.00,64.1,43.2,2012-09-27
3462,4.2,0.00,68.0,46.2,2012-09-28
3463,4.2,0.00,69.8,50.9,2012-09-29


In [275]:
data = data.set_index('Date')

In [276]:
data

,runoff_obs,precip,tmax,tmin
Date,,,,
2002-10-01,2.7,0.13,57.4,34.6
2002-10-02,2.7,0.97,38.4,29.7
2002-10-03,2.8,0.04,41.2,31.7
2002-10-04,2.8,0.00,52.9,34.0
2002-10-05,2.6,0.00,50.5,34.8
...,...,...,...,...
2012-09-26,4.4,0.00,59.2,41.0
2012-09-27,4.3,0.00,64.1,43.2
2012-09-28,4.2,0.00,68.0,46.2


In [277]:
data_values = data.values

x = data_values[:,1:]
y = data_values[:,0]

In [278]:
from sklearn.svm import SVR
clf = SVR(kernel='rbf', gamma=0.001, C=1.0, epsilon=0.1)
print(clf)

SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma=0.001,
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)


In [279]:
clf.fit(x, y, sample_weight=None)

SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma=0.001,
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [280]:
clf.score(x, y, sample_weight=None)

0.29888189291756107

In [281]:
from sklearn.ensemble import GradientBoostingRegressor
clf1 = GradientBoostingRegressor(loss='ls', learning_rate=0.1, n_estimators=100,
       subsample=1.0, criterion='friedman_mse', min_samples_split=2, min_samples_leaf=1,
       min_weight_fraction_leaf=0.0, max_depth=3, min_impurity_decrease=0.0, min_impurity_split=None,
       init=None, random_state=None, max_features=None, alpha=0.9, verbose=0, max_leaf_nodes=None, 
       warm_start=False, presort='auto', validation_fraction=0.1, tol=0.0001)
print(clf1)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
                          learning_rate=0.1, loss='ls', max_depth=3,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=100,
                          n_iter_no_change=None, presort='auto',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)


In [282]:
clf1.fit(x, y, sample_weight=None)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
                          learning_rate=0.1, loss='ls', max_depth=3,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=100,
                          n_iter_no_change=None, presort='auto',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

In [283]:
clf1.score(x, y, sample_weight=None)

0.49649066351945553

In [236]:
from sklearn.model_selection import cross_val_predict, KFold, cross_val_score

In [237]:
pred = cross_val_predict(clf1, x, y, cv=6)
print(pred)

[4.60093693 2.03673965 2.93711086 ... 5.30001692 7.69263462 8.03490789]


In [238]:
from sklearn import metrics
Rsquared = metrics.r2_score(y, pred)
print(Rsquared)

0.03147045752392985


In [239]:
scores = cross_val_score(clf1, x, y, cv=6, scoring='r2')
print(scores)

[-1.33881081  0.01195994 -0.19402528 -0.28171045  0.11863819  0.07551197]


In [240]:
# this is for testing
print(cross_val_score(clf1, x, y, cv=6, scoring='r2'))

[-1.357579    0.01195994 -0.19180269 -0.28171045  0.11980246  0.07494181]


In [241]:
scores = []
kf = KFold(n_splits=5, random_state=None,  shuffle=False) 
for train_index, test_index in kf.split(x):  
    
    print("Train Index: ", train_index, "\n")
    print("Test Index: ", test_index)
    
x_train, x_test, y_train, y_test = x[train_index], x[test_index], y[train_index], y[test_index]
clf1.fit(x_train, y_train) 
scores.append(clf1.score(x_test, y_test))

Train Index:  [ 731  732  733 ... 3650 3651 3652] 

Test Index:  [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197
 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215
 216 217 218 219 220 221 222 223 224 225 226 227 228 229 23

In [242]:
print(np.mean(scores))

0.05920881555859625
